In [25]:
from tkinter import filedialog
import tkinter as tk
import PIL.Image, PIL.ImageTk
import numpy as np
import cv2
import math
import random
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pylab as plt
from matplotlib import gridspec

In [26]:
# global variables
MARGIN = 10  # px
MAXDIM = 500

class Load_Image():
    def __init__(self, window, window_title, bg_image='bg.jpeg'):
        self.window = window
        self.window.title(window_title)
        self.var = tk.IntVar()
        self.file_path = None

        # Load the background image using OpenCV
        self.bg_img = cv2.cvtColor(cv2.imread(bg_image), cv2.COLOR_BGR2RGB)
        self.height, self.width, no_channels = self.bg_img.shape
        
        # Use PIL (Pillow) to convert the NumPy ndarray to a PhotoImage
        self.photoBG = PIL.ImageTk.PhotoImage(image=PIL.Image.fromarray(self.bg_img))
        
        # Create a FRAME that can fit the images, BLUE
        self.frame1 = tk.Frame(self.window, width=self.width, height=self.height, bg='white')
        self.frame1.pack(fill=tk.BOTH)
        
        # Create a Cavas for the background image
        self.canvas0 = tk.Canvas(self.frame1, width=self.width, height=self.height, bg='white')
        self.canvas0.pack(side=tk.LEFT)
        
        # Add a PhotoImage to the Canvas (original)
        self.canvas0.create_image(self.width//2,  self.height//2, image=self.photoBG)
        
        # Display the button for browing image file
        self.button = tk.Button(self.frame1, text ="Browse", command=self.browse_image, bg="yellow",fg="black")
        self.button.place(x=720, y=336, anchor="c")
        
        # Wait until user select an image from file. 
        self.button.wait_variable(self.var)
        if self.file_path is not None:
            window.destroy()
        
        self.window.mainloop()
        
    def __del__(self):
        print("Object deleted")

    #Browse Image File
    def browse_image(self):
        file = filedialog.askopenfile(mode='r', title="Choose an image file")
        if file:
            self.file_path = file.name
            self.var.set(1)


In [27]:
class App():
    def __init__(self, window, window_title, image_path="lena_color.jpeg"):
        self.window = window
        self.window.title(window_title)
        self.image_path = image_path
        
        # Load an image using OpenCV
        self.cv_img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

        # Get the image dimensions (OpenCV stores image data as NumPy ndarray)
        self.height, self.width, no_channels = self.cv_img.shape
        
        # If the image dimension is greater than canvas, resize to fit into the canvas
        if max(self.width, self.height) > MAXDIM:
            if self.width/self.height >= 1:
                self.cv_img = cv2.resize(self.cv_img, (MAXDIM, int(self.height*MAXDIM/self.width)))
            else:
                self.cv_img = cv2.resize(self.cv_img, (int(self.width*MAXDIM/self.height), MAXDIM))
            self.height, self.width, no_channels = self.cv_img.shape
                
        self.NEWcv_img = self.cv_img.copy()  # for recursive processing
        self.NEWcv_img_modify = None
        
        ''' Image Display Related Code'''
        # Create a FRAME that can fit the images, BLUE
        self.frame1 = tk.Frame(self.window, width=MAXDIM*2+MARGIN, height=MAXDIM*2, bg='blue')
        self.frame1.pack(fill=tk.BOTH)
        
        # Create a CANVAS for original image, YELLOW
        self.canvas0 = tk.Canvas(self.frame1, width=MAXDIM, height=MAXDIM+(3*MARGIN), bg='white')
        self.canvas0.pack(side=tk.LEFT)
        
        # Create a CANVAS for changing image, ORANGE
        self.canvas1 = tk.Canvas(self.frame1, width=MAXDIM, height=MAXDIM+(3*MARGIN), bg='white')
        self.canvas1.pack(side=tk.LEFT)
        
        # Use PIL (Pillow) to convert the NumPy ndarray to a PhotoImage
        self.photoOG = PIL.ImageTk.PhotoImage(image=PIL.Image.fromarray(self.cv_img))
        self.photo = PIL.ImageTk.PhotoImage(image=PIL.Image.fromarray(self.cv_img))
        
        # Add a PhotoImage to the Canvas (original)
        self.canvas0.create_image(MAXDIM//2, MAXDIM//2, image=self.photoOG)
        
        # Add a PhotoImage to the Canvas (changing effects)
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        
        # Write labels for both images, font/size can be changed
        self.canvas0.create_text(MAXDIM//2, MAXDIM+(2*MARGIN),font="Tahoma 20",text="Original Photo")
        self.canvas1.create_text(MAXDIM//2, MAXDIM+(2*MARGIN),font="Tahoma 20",text="Modified Photo")
        
# ##############################################################################################
# ################################   PARAMETER TOOLBAR   #######################################
# ##############################################################################################

        # Create a FRAME that can fit the features
        self.frame2 = tk.Frame(self.window, width=self.width, height=self.height//2.3, bg='grey')
        self.frame2.pack(side=tk.BOTTOM, fill=tk.BOTH)
        
        # Create a SCALE that lets the user use "Pop Art Effect"
        self.scl_PopArt_effect = tk.Scale(self.frame2, from_=1, to=10, orient=tk.HORIZONTAL, showvalue=0,
                command = self.PopArt_effect, length=400, sliderlength=20, label="Pop Art (Segmentation Effect)", font="Tahoma 10")
        self.scl_PopArt_effect.place(relx=0.02, rely=0.1, relwidth=0.45, relheight=0.2)
        
        # Create a SCALE that lets the user "glitch Art"
        self.scl_glitch_art = tk.Scale(self.frame2, from_=1, to=50, orient=tk.HORIZONTAL, showvalue=0,
                command = self.Glitch_art, length=400, sliderlength=20, label="Glitch art", font="Tahoma 10")
        self.scl_glitch_art.place(relx=0.5, rely=0.1, relwidth=0.45, relheight=0.2)
        
        # Create a SCALE that lets the user use "pixelation Effect"
        self.scl_Pixelation_effect = tk.Scale(self.frame2, from_=1, to=20, orient=tk.HORIZONTAL, showvalue=0,
                command = self.Pixelation, length=400, sliderlength=20, label="Pixel Art Effect", font="Tahoma 10")
        self.scl_Pixelation_effect.place(relx=0.02, rely=0.32, relwidth=0.45, relheight=0.2)
        
        # Create a SCALE that lets the user use "Dots Effect"
        self.scl_Dots_effect = tk.Scale(self.frame2, from_=1, to=30, orient=tk.HORIZONTAL, showvalue=0,
                command = self.Dots, length=400, sliderlength=20, label="Dot Art Effect", font="Tahoma 10")
        self.scl_Dots_effect.place(relx=0.5, rely=0.32, relwidth=0.45, relheight=0.2)
        
        # Create a SCALE that lets the user "black & yello contrast" effect
        self.scl_black_contrast = tk.Scale(self.frame2, from_=1, to=50, orient=tk.HORIZONTAL, showvalue=0,
                command = self.Black_contrast, length=400, sliderlength=20, label="Black and Yellow Contrast", font="Tahoma 10")
        self.scl_black_contrast.place(relx=0.02, rely=0.54, relwidth=0.45, relheight=0.2)
        
        # Create a SCALE that lets the user "cartoon effect"
        self.scl_cartoon_effect = tk.Scale(self.frame2, from_=1, to=10, orient=tk.HORIZONTAL, showvalue=0,
                command = self.Cartoon_effect, length=400, sliderlength=20, label="Cartoon Effect", font="Tahoma 10")
        self.scl_cartoon_effect.place(relx=0.5, rely=0.54, relwidth=0.45, relheight=0.2)
        
        # Create a Radio Button that lets the user choose "Artistic style"
        self.label_artistic_style = tk.Label(self.frame2, text="(Artistic Style Transfer) Choose an artist \n\n")
        self.label_artistic_style.config(font=("Tahoma", 13))
        self.label_artistic_style.pack()
        self.label_artistic_style.place(relx=0.02, rely=0.76, relwidth=0.93, relheight=0.20)    
        
        self.val = tk.StringVar()
        artistic_styles = ['Yayoi_Kusama','Keith_Haring','Lichtenstein','Kandinsky','Kanagawa','VanGogh']
        xelx_index = 0.02
        
        for artist in artistic_styles:
            button = tk.Radiobutton(self.frame2, text=artist, padx=20, variable=self.val, value=artist, command=self.Artistic_style_transfer)
            button.config(font=("Tahoma", 12))
            button.pack(expand = True, side='left')
            button.place(relx=xelx_index, rely=0.86, relwidth=0.10, relheight=0.10)
            xelx_index = xelx_index+0.15
            button.deselect()
        
        self.window.mainloop()
        
##############################################################################################
#################################  CALLBACK FUNCTIONS  #######################################
##############################################################################################
    '''########################### Set scales to default  ###########################'''
    #If one function is work, set other scales to default 
    def set_scls_to_default(self, func):
        if func is not "PopArt_effect": self.scl_PopArt_effect.set(1)
        if func is not "Dots": self.scl_Dots_effect.set(1)
        if func is not "Pixelation": self.scl_Pixelation_effect.set(1)
        if func is not "Black_contrast": self.scl_black_contrast.set(1)
        if func is not "Cartoon_effect": self.scl_cartoon_effect.set(1)
        if func is not "Glitch_art": self.scl_glitch_art.set(1)
        #if func is not "Artistic_style_transfer": self.scl_glitch_art.set(1) # TO DO
            
    '''###########################  Reduce color palette ###########################'''
    # Reduced the number of color used in an image. 
    def reduce_color_palette(self, total_color=20, colormap=False):
        gaussian = cv2.blur(self.NEWcv_img, (5, 5)) # Gaussian Blur 
        data = np.float32(gaussian).reshape((-1, 3)) # Transform the image
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 0.001) # Determine criteria
        
        # Implementing K-Means
        ret, label, center = cv2.kmeans(data, total_color, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
        
        # pick the color for centers if colormap is true (assign different color)
        if colormap is True:
            for i in range(len(center)):
                # No need to assgin the color manually using sin, coine value
                center[i] = np.rint([(math.sin(i*0.3)+1)*122,(math.sin(i*0.6)+1)*122,(math.sin(i*0.9)+1)*122]) 
        
        center = np.uint8(center)
        result = center[label.flatten()]        
        return result.reshape(self.NEWcv_img.shape)
    
    '''###########################  Increase Edge ###########################'''
    # Detect edge and change the edge size. 
    def increase_edge(self, img, line_size, blur_value=9):
        line_size = line_size*2+1 #line size (odd number, minimum is 3)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray_blur = cv2.medianBlur(gray, blur_value) # Median Blur to reduce salt & pepper noise
        return cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, line_size, blur_value)
        
    '''###########################  Pop Art Effect  ###########################'''
    # Callback for the "Pop Art Effect" Scale 
    def PopArt_effect(self, k):
        self.set_scls_to_default("PopArt_effect") #Set other scale to default(1)
        k = self.scl_PopArt_effect.get()  # get value from the corresponding scale
        
        # Reduce Color Palette and change the color using colormap
        reducedColorPalette = self.reduce_color_palette(total_color=-k+12, colormap=True)

        self.NEWcv_img_modify = reducedColorPalette
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(self.NEWcv_img_modify))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        
    '''###########################  Dot Art effect ###########################'''
    # Callback for the "Dot arts" Scale
    def Dots(self, k):
        self.set_scls_to_default("Dots") #Set other scale to default(1)
        k = self.scl_Dots_effect.get()  # get value from the corresponding scale
        
        # Set different color when k is updated. No need to manually select colors. 
        dots_colour = np.rint([(math.sin(k*0.3)+1)*122,(math.sin(k*0.6)+1)*122,(math.sin(k*0.9)+1)*122]) 
        background_colour = np.rint([(math.sin(k*0.4)+1)*122,(math.sin(k*0.8)+1)*122,(math.sin(k)+1)*122])
        
        max_dots = 100 # set the max dots (on the longest side of the image)
        grayImage = cv2.cvtColor(self.NEWcv_img, cv2.COLOR_BGR2GRAY) # Change to gray image
        height, width = grayImage.shape # Get dimensions
        # down size to number of dots (longest side of imge = max dots)
        if self.width >= self.height:
            downsized_image = cv2.resize(grayImage, (max_dots, int(height*max_dots/width)))
        else:
            downsized_image = cv2.resize(grayImage, (int(width*max_dots/height), max_dots))

        height, width = downsized_image.shape # Get dimensions of new image
        multiplier = int(self.height/height) # set how big we want our final image to be
        
        # set the padding value so the dots start in frame (rather than being off the edge
        padding = int(multiplier/2)
        # create canvas containing just the background colour
        blank_image = np.full(((self.height),(self.width),3), background_colour, dtype=np.uint8)
        # run through each pixel and draw the circle on our blank canvas
        for y in range(0,height):
            for x in range(0,width):
                cv2.circle(blank_image,(((x*multiplier)+padding),((y*multiplier)+padding)), int((0.6 * multiplier) * ((255-downsized_image[y][x])/255)), dots_colour, -1)

        self.NEWcv_img_modify = blank_image # Final our image
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(self.NEWcv_img_modify))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)       

    '''###########################  Glitch Art ###########################'''
    # Callback for the "Glitch art" Scale 
    def Glitch_art(self, k):
        self.set_scls_to_default("Glitch_art") #Set other scale to default(1)
        k = self.scl_glitch_art.get()
        img = self.NEWcv_img.copy()
        b,g,r = cv2.split(img)
    
        # Randomly generate horizontal glitch line 
        for i in range(k):
            width_size = random.randrange(int(self.width/2))
            column = random.randrange(self.width-width_size)
            height_size = random.randrange(3)
            row = random.randrange(self.height-height_size)
            img[row:row+height_size, column:column+width_size,] = img[row,column,:]        
        
        #Tearing effect. Shift some parts of image to right (k is the size of block)
        shift = int(0.5*k)
        start_row = int(img.shape[0]*0.2)
        end_row = int(img.shape[0]*(k*0.01)+0.2)
        for column in range(img.shape[1]):
            column = img.shape[1]-1-column #access column from right side to prevent overriding. 
            if column+shift < img.shape[1]: # Shift Image to right 
                img[start_row:end_row, column+shift,] = img[start_row:end_row, column,] 
        
        #Shift blue channel to left, k is shifting size. 
        shift = k
        for column in range(img.shape[1]):
            if column%2 is 0:
                if column+shift < img.shape[1]: # Shift Blue channel to left 
                    img[:, column, 0] = b[:,column+shift]
                else: # Shift blue channel to right side of image if it exceeds the boundary
                    img[:, column, 0] = b[:,img.shape[1]-column-shift]
        
        #Shift red channel to upward, k is shifting size. 
        shift = int(0.5*k)
        for row in range(img.shape[0]):
            if row%3 is 0:
                if row+shift < img.shape[0]: # Shift red channel to up 
                    img[row,:,1] = g[row+shift,:]
                else: # Shift red channel to bottom side of image if it exceeds the boundary
                    img[row,:,1] = g[img.shape[0]-row-shift,:]        
                
        self.NEWcv_img_modify = img
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(self.NEWcv_img_modify))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
    
    '''#################################  Pixelation effect  ###############################'''
    # Callback for the "Pixelation" Scale 
    def Pixelation(self, k):
        self.set_scls_to_default("Pixelation") #Set other scale to default(1)
        k = self.scl_Pixelation_effect.get()  # get value from the corresponding scale
        
        # Reduce the color palette. K adjust the number of color used in the image. 
        reduced_color_palette_img = self.reduce_color_palette(total_color= int(0.2*-k)+8, colormap=False)
        height, width = reduced_color_palette_img.shape[:2]
        
        # "pixelated" size
        w, h = (-5*k+105, -5*k+105) 
        # Resize input to "pixelated" size
        resized = cv2.resize(reduced_color_palette_img, (w, h), interpolation=cv2.INTER_LINEAR)
        
        self.NEWcv_img_modify = cv2.resize(resized, (width, height), interpolation=cv2.INTER_NEAREST)
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(self.NEWcv_img_modify))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
    
    '''##############################  Black and Yellow Contrast  ############################'''
    # Callback for the "Black and Yellow contrast" Scale 
    def Black_contrast(self, k):
        self.set_scls_to_default("Black_contrast") #Set other scale to default(1)
        k = self.scl_black_contrast.get()  # get value from the corresponding scale
        
        gaussian = cv2.blur(self.NEWcv_img, (5, 5)) # Gaussian filter
        grayImage = cv2.cvtColor(gaussian, cv2.COLOR_BGR2GRAY) # Change to gray image
        (thresh, blackAndWhiteImage) = cv2.threshold(grayImage, 2*k+50, 255, cv2.THRESH_BINARY)
        colorImage = cv2.cvtColor(blackAndWhiteImage, cv2.COLOR_GRAY2RGB) # Change to gray image
        colorImage[np.where((colorImage==[255,255,255]).all(axis=2))] = [255,230,0] #Change background to yellow 
        
        self.NEWcv_img_modify = colorImage
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(self.NEWcv_img_modify))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        
    '''#################################  Cartoon Effect  ###############################'''
    # Callback for the "Cartoon_effect" Scale 
    def Cartoon_effect(self, k):
        self.set_scls_to_default("Cartoon_effect") #Set other scale to default(1)
        k = self.scl_cartoon_effect.get()  # get value from the corresponding scale
        
        # Use bilateral filter, k adjust affecting color and area(sigma) used in bilateral filter
        image = cv2.bilateralFilter(self.NEWcv_img, d=30*k, sigmaColor=5*k+30, sigmaSpace=30*k)
        
        # Adjust edge size. k adjust edge size
        edges = self.increase_edge(image, int(k/3)+3)
        
        # Color Palette image + Edge image 
        self.NEWcv_img_modify = cv2.bitwise_and(image, image, mask=edges)
        
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(self.NEWcv_img_modify))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
    
    '''#################################  Artistic Style Transfer ###############################'''
    # Callback for the "Artistic Style Transfer" radio button 
    def Artistic_style_transfer(self):
        self.set_scls_to_default("Artistic_style_transfer")
        artist = str(self.val.get())
        
        style_image_path = None
        if artist == "Yayoi_Kusama":
            style_image_path = '1_YayoiKusama.jpeg'
        elif artist == 'Keith_Haring':
            style_image_path = '2_KeithHaring.jpeg'
        elif artist == 'Lichtenstein':
            style_image_path = '3_Lichtenstein.jpeg'
        elif artist == 'Kandinsky':
            style_image_path = '4_Kandinsky.jpeg'
        elif artist == 'Kanagawa':
            style_image_path = '5_Kanagawa.jpeg'
        elif artist == 'VanGogh':
            style_image_path = '6_VanGogh.jpeg'
            
        # Load TF Hub module.
        hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
        hub_module = hub.load(hub_handle)
    
        # Load and convert to float32 numpy array, add batch dimension, and normalize to range [0, 1].
        img_size = (self.height, self.width)
        content_image = tf.io.decode_image(tf.io.read_file(self.image_path),channels=3, dtype=tf.float32)[tf.newaxis, ...]
        content_image = tf.image.resize(content_image, img_size, preserve_aspect_ratio=True)
        style_image = tf.io.decode_image(tf.io.read_file(style_image_path),channels=3, dtype=tf.float32)[tf.newaxis, ...]
        style_image = tf.image.resize(style_image, img_size, preserve_aspect_ratio=True)
        
        #The signature of this hub module for image stylization is:
        outputs = hub_module(content_image, style_image)
        stylized_image = outputs[0][0]
        stylized_image = np.array(stylized_image * 255).astype(np.uint8)
        
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(stylized_image))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        

In [ ]:
##############################################################################################
# Create a window and pass it to the Load_Image object
Home = Load_Image(tk.Tk(), "** Welcome to Art Creator **")
file_path = Home.file_path
del Home

# Create a window and pass it to the Application object
Application = App(tk.Tk(), "** Art Creator **", file_path)